In [4]:
import numpy as np
import pandas as pd
import my_globals

In [5]:
from get_sub_dataset import get_sub_dataset

size = 100,
random_seed = 434
get_sub_dataset(size = size, random_seed=random_seed)
data = pd.read_csv(my_globals.DATA_DIR + f"/twitter_seed{random_seed}.csv")
data

,target,ids,date,flag,user,text
0,0,2181073436,Mon Jun 15 11:19:51 PDT 2009,NO_QUERY,AshleeStevens,Wisdom teeth suck
1,0,2328857491,Thu Jun 25 10:04:09 PDT 2009,NO_QUERY,poshmama,"R.I.P. Farrah Fawcett, I will miss your beauti..."
2,0,2014076278,Wed Jun 03 00:23:22 PDT 2009,NO_QUERY,tinydeww,@modwheelmood orlando sometime? cant drive to ...
3,4,1685659526,Sun May 03 01:25:42 PDT 2009,NO_QUERY,SamyRox,"just had a pulled chicken with pesto, swiss ch..."
4,4,1989032538,Mon Jun 01 00:07:36 PDT 2009,NO_QUERY,petrarco,@msdivamom Good night and sweet dreams.
...,...,...,...,...,...,...
95,0,2296279150,Tue Jun 23 08:59:23 PDT 2009,NO_QUERY,momojos,can't find a date for step 1 ANYWHERE. and i'...
96,4,2071407698,Sun Jun 07 19:20:54 PDT 2009,NO_QUERY,awaitinserenity,@stargirlreads aw girl you need to have that o...
97,4,1992956361,Mon Jun 01 09:35:48 PDT 2009,NO_QUERY,witnessamiracle,"@dousethelights Heyy! How're you doing? Oh, I..."
98,0,2006459542,Tue Jun 02 11:11:58 PDT 2009,NO_QUERY,liddyx,"@xnuny yeeeah, a lot!"
